In [1]:
using Random
using Distributions
using DataFrames, CSV
using FLoops

# compile local files
include("statistics.jl")
include("simulation.jl")
include("data.jl")
include("util.jl")

using .TestStatistics
using .Simulation
using .Data

In [15]:
"""
CONFIG
"""
alpha = 0.05

# data
B  = 2    # num. coverage probabilities per boxplot
S  = 1800   # num. samples per coverage probability
nx = 8      # size of group 1
ny = 8      # size of group 2
px, py = partition(nx, ny)
dtype = Float32

# distribution settings
Random.seed!(123)

distrTypeX = Normal
shapeX = dtype.(rand(Uniform(1, 5), B))
scaleX = 4 * shapeX
paramsX = (shapeX, scaleX)

distrTypeY = Normal
shapeY = dtype.(rand(Uniform(1, 5), B))
scaleY = 4 * shapeY
paramsY = (shapeY, scaleY)

(Float32[4.5635147, 1.7636268], Float32[18.254059, 7.0545073])

In [ ]:
"""
distrTypeX = Exponential
lambdaX = dtype.(rand(Uniform(1, 20), B))
paramsX = (lambdaX,)

distrTypeY = Exponential
lambdaY = dtype.(rand(Uniform(1, 20), B))
paramsY = (lambdaY,)


distrTypeX = LogNormal
muX = dtype.(rand(Uniform(0, 1), B))
sdX = 2 * muX
paramsX = (muX, sdX)

distrTypeY = LogNormal
muY = dtype.(rand(Uniform(0, 1), B))
sdY = 2 * muY
paramsY = (muY, sdY)


distrTypeX = Gamma
shapeX = dtype.(rand(Uniform(1, 5), B))
scaleX = 4 * shapeX
paramsX = (shapeX, scaleX)

distrTypeY = Gamma
shapeY = dtype.(rand(Uniform(1, 5), B))
scaleY = 4 * shapeY
paramsY = (shapeY, scaleY)
"""

In [16]:
# generate data
Random.seed!(123)
x, y, wide, narrow, deltas, distrX, distrY = Data.generateData(B, S, nx, ny, true,
                                                          distrTypeX, paramsX,
                                                          distrTypeY, paramsY)
@show size(x)
@show size(y)
@show size(wide)
@show size(narrow)
@show deltas[1]
@show distrX[1]
@show distrY[1]
x[1:5]

size(x) = (28800,)
size(wide) = (3600,)
size(x) = (8, 1800, 2)
size(y) = (8, 1800, 2)
size(wide) = (1800, 2)
size(narrow) = (1800, 2)
deltas[1] = -1.4786596f0
distrX[1] = Normal{Float32}(μ=3.084855f0, σ=12.33942f0)
distrY[1] = Normal{Float32}(μ=4.5635147f0, σ=18.254059f0)


5-element Vector{Float32}:
  -4.883087
 -14.9708185
 -16.949474
   0.39899373
   9.158881

In [4]:
# wrap constant arguments into an iterable object
# this allows each batch to have local access

struct Args
    px::AbstractMatrix{Int}
    py::AbstractMatrix{Int}
    pooled::Bool
    alpha::Float32
    alt_lo::Alternative
    alt_hi::Alternative
end

Base.Broadcast.broadcastable(obj::Args) = Ref(obj)  # make objects of this type iterable

In [56]:
basesize = ceil(Int, B / Threads.nthreads())

@time for isTwoSided in [true, false]
    alpha_temp = alpha
    alt_lo = alt_hi = twoSided
    
    if !isTwoSided
        alpha_temp = alpha / 2
        alt_lo = greater
        alt_hi = smaller
    end
        
    for pooled in [true, false]
        args = Args(px, py, pooled, alpha_temp, alt_lo, alt_hi)
        
        
        results = Vector{Any}(undef, B)
        
        @floop ThreadedEx(basesize = 1) for i = 1:B
            @floop ThreadedEx() for j = 1:S
                @inbounds a, b = Simulation.permInterval(x[:,j,i], y[:,j,i], wide[j,i], narrow[j,i], deltas[i], args)
                @reduce() do (coverage = 0; a), (width = 0.0f0; b)
                    coverage += a
                    width += b
                end
            end
            results[i] = (coverage / S, width / S)
        end
        
        # convert results to DataFrame
        probs  = [i for (i, _) in results]
        widths = [j for (_, j) in results]
        df = DataFrame(prob=probs, width=widths, distrX=distrX, distrY=distrY)

        # save DataFrame as .csv
        filename = (isTwoSided ? "twoSided" : "oneSided") * "_" *
                   (pooled ? "" : "un") * "pooled" * "_" * string(args.alpha) * ".csv"
        
        #CSV.write("../results/" * filename, df)
        CSV.write(filename, df)
    end
end

 13.308303 seconds (12.61 M allocations: 71.634 GiB, 16.91% gc time, 1.12% compilation time)


In [17]:
args = Args(px, py, true, 0.05, twoSided, twoSided)
results = Vector{Any}(undef, B)

@time @floop ThreadedEx(basesize = 1) for i = 1:B
    @floop ThreadedEx() for j = 1:S
        @inbounds a, b = Simulation.permInterval(x[:,j,i], y[:,j,i], wide[j,i], narrow[j,i], deltas[i], args)
        @reduce() do (coverage = 0; a), (width = 0.0f0; b)
            coverage += a
            width += b
        end
    end
    results[i] = (coverage / S, width / S)
end

results

  7.573892 seconds (7.09 M allocations: 42.486 GiB, 20.28% gc time, 2.17% compilation time)


2-element Vector{Any}:
 (0.9494444444444444, 32.448074f0)
 (0.9461111111111111, 22.329578f0)